# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21097014


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:05<01:17,  2.77s/it]

Rendering models:  13%|█▎        | 4/30 [00:05<00:51,  1.97s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:46,  1.86s/it]

Rendering models:  20%|██        | 6/30 [00:07<00:32,  1.33s/it]

...

Rendering models:  73%|███████▎  | 22/30 [00:08<00:01,  5.54it/s]

Rendering models:  80%|████████  | 24/30 [00:09<00:00,  6.36it/s]

Rendering models:  87%|████████▋ | 26/30 [00:09<00:00,  7.05it/s]

Rendering models:  93%|█████████▎| 28/30 [00:09<00:00,  8.53it/s]

Jnursssmith5263                       0.000588
not-logged-in-40ca25f4608b8f65e1a8    0.001497
Idax98                                0.000178
taysish58                             0.000195
mwalsh868                             0.021609
djfido                                0.001514
thiagoxxxx                            0.000286
framosuw                              0.000182
miyosolange                           0.000240
ElisabethB                            0.000193
not-logged-in-5f30b2bb9b7f9d49da41    0.004347
not-logged-in-9cdd68bb509752635e0b    0.000277
not-logged-in-92e92166c0411a000b1a    0.000315
tominas                               0.000273
Lavadude                              0.018321
BRutherford                           0.000716
crush202020                           0.000550
not-logged-in-b870acc91e92db64a19e    0.021134
ChungusStates                         0.010062
TheEpicPrimius123                     0.000989
AspenTreez                            0.000400
slambier     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())